# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [1]:
!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 52kB/s 
     |████████████████████████████████| 3.8MB 34.5MB/s 
     |████████████████████████████████| 450kB 31.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=90da645cddcdcbfbc90e5d50ec4a2f92157eecf586cc02cff571cf43b6115efa
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [2]:
#Import Libraries
import pandas as pd
import numpy as np


## Get Required Files from Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
#Set your project path 
project_path =  "/content/drive/My Drive/Residency-10/"

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [5]:
df = pd.read_json(project_path + "Sarcasm_Headlines_Dataset_drive.json",lines=True)

In [6]:
df.shape

(26709, 3)

In [7]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
#There are 3 cols  with 26709 records.

## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [8]:
#Drop col article_link from the dataset 
df.drop("article_link",axis=1, inplace=True)

In [9]:
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [10]:
#Creating a column which will store the length count of its corresponding "headline" col row.
df['len'] = df['headline'].str.split(" ").str.len()

In [12]:
df.head()

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14
3,"boehner just wants wife to listen, not come up...",1,13
4,j.k. rowling wishes snape happy birthday in th...,0,11


In [17]:
df.describe()

,is_sarcastic,len
count,26709.000000,26709.000000
mean,0.438953,9.847842
std,0.496269,3.172099
min,0.000000,2.000000
25%,0.000000,8.000000
50%,0.000000,10.000000
75%,1.000000,12.000000
max,1.000000,39.000000


#**## Modelling**

## Import required modules required for modelling.

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [19]:
max_features = 10000
maxlen = 39              ## Add your max length here ##
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [20]:
tokeniser = Tokenizer(num_words=max_features)
tokeniser.fit_on_texts(list(df.headline))

# Define X and y for your model.

In [21]:
X = tokeniser.texts_to_sequences(df['headline'])
print("X[0]- Without padding : ", X[0])
X = pad_sequences(X, maxlen = maxlen)

y = np.asarray(df['is_sarcastic'])

print("Number of Samples :", len(X))
print("X[0]- with padding : ", X[0])
print("Number of Labels : ", len(y))
print(y[0])

X[0]- Without padding :  [307, 678, 3336, 2297, 47, 381, 2575, 5, 2576, 8433]
Number of Samples : 26709
X[0]- with padding :  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  307  678 3336 2297   47  381 2575    5 2576 8433]
Number of Labels :  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [22]:
voc_size = len(tokeniser.word_index) + 1
print("Vocabulary size : ", voc_size)

Vocabulary size :  29657


#**## Word Embedding**

## Get Glove Word Embeddings

In [23]:
glove_file = project_path + "glove.6B.zip" 

In [24]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [25]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [26]:
embedding_matrix = np.zeros((voc_size, 200))

for word, i in tokeniser.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [27]:
### Embedding layer for hint 
model = Sequential()
model.add(Embedding(voc_size, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         5931400   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         336896    
_________________________________________________________________
dense (Dense)                (None, None, 40)          10280     
_________________________________________________________________
dropout (Dropout)            (None, None, 40)          0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 20)          820       
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 20)          0         
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           2

# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [29]:
batch_size = 100
epochs = 25
## Add your code here ##
model.fit(X,y,batch_size=batch_size, epochs=epochs, validation_split=0.2)


Train on 21367 samples, validate on 5342 samples
Epoch 1/25
21367/21367 [==============================] - 103s 5ms/sample - loss: 0.4829 - accuracy: 0.7711 - val_loss: 0.3555 - val_accuracy: 0.8402
Epoch 2/25
21367/21367 [==============================] - 100s 5ms/sample - loss: 0.2836 - accuracy: 0.8937 - val_loss: 0.3285 - val_accuracy: 0.8595
Epoch 3/25
21367/21367 [==============================] - 100s 5ms/sample - loss: 0.1938 - accuracy: 0.9315 - val_loss: 0.3545 - val_accuracy: 0.8623
Epoch 4/25
21367/21367 [==============================] - 100s 5ms/sample - loss: 0.1347 - accuracy: 0.9533 - val_loss: 0.4502 - val_accuracy: 0.8551
Epoch 5/25
21367/21367 [==============================] - 100s 5ms/sample - loss: 0.0932 - accuracy: 0.9685 - val_loss: 0.6410 - val_accuracy: 0.8528
Epoch 6/25
21367/21367 [==============================] - 100s 5ms/sample - loss: 0.0627 - accuracy: 0.9794 - val_loss: 0.8486 - val_accuracy: 0.8497
Epoch 7/25
21367/21367 [===========================

In [ ]:
#We have achieved highest validation accuracy of 86.23%